In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

In [2]:
page = requests.get('https://covid.icmr.org.in/index.php/testing-facilities/15-government-laboratories/01')
page.status_code

404

In [3]:
hotspot = []
pages = []
for i in range(0,300):
    page = requests.get('https://covid.icmr.org.in/index.php/testing-facilities/15-government-laboratories/' + str(i))
    if (page.status_code==200):
        hotspot.append(i)
        pages.append(page)
        #print(i)

In [4]:
print(len(hotspot))
print('min: ',min(hotspot))
print('max: ',max(hotspot))

181
min:  3
max:  218


In [5]:
labs_location = []
CentreName = []
for page in pages:
    soup = BeautifulSoup(page.content, 'html.parser')
    all_hotspots = soup.find_all("div" ,class_ ='hotspots-description')
    labs_location.append(all_hotspots[0].string)
    
    name = soup.find_all("h2",class_='componentheading')
    CentreName.append(name[0].string)

In [6]:
print(labs_location[0][7:-9])
CentreName[0].string[6:-21]

Post Bag 13, Dollygunj, Port Blair, Andaman and Nicobar Islands 744103


'Regional Medical Research Centre (ICMR), Port Blair '

In [7]:
addresses = []
title = []
for i in range(len(labs_location)):
    if labs_location[i] is not None:
        addresses.append(labs_location[i].string[7:-9])
    else :
        addresses.append(labs_location[i])
    title.append(CentreName[i].string[6:-21])

In [8]:
hotspot_arr = np.array(hotspot).reshape(len(hotspot),1)
address = np.array(addresses).reshape(len(hotspot),1)
LabName = np.array(title).reshape(len(hotspot),1)
data = np.concatenate((hotspot_arr,LabName,address),axis=-1)
df = pd.DataFrame(data,columns=['HotspotID' ,'LabName','Addresses'])
df

,HotspotID,LabName,Addresses
0,3,"Regional Medical Research Centre (ICMR), Port ...","Post Bag 13, Dollygunj, Port Blair, Andaman an..."
1,4,Sri Venkateswara Institute of Medical Sciences...,Sri Venkateswara Institute of Medical Sciences...
2,5,"Sidhartha Medical College, Vijayawada","Siddhartha Medical College, Ring Road, Gunadal..."
3,6,"Government Medical College, Anantapur","Government Medical College, Anantapur, Rahamat..."
4,8,"Rangaraya Medical College, Kakinada","Rangaraya Medical College, Pithampuram Road, K..."
5,11,"ICMR-Regional Medical Research Centre, NE Regi...","ICMR-Regional Medical Research Centre, Dibruga..."
6,12,"Gauhati Medical College, Guwahati","Guwahati Narakasur Hilltop, Bhangagarh, Guwaha..."
7,13,"Silchar Medical College, Silchar","Silchar Medical College, Ghungoor, Silchar 788..."
8,14,"Jorhat Medical College, Jorhat","Jorhat Medical College, Kushal Konwar Path, Ba..."
9,16,"Fakhruddin Medical College, Barpeta","Barpeta-Hospital-Jania Rd, Joti Gaon, Assam 78..."


In [9]:
nonedata = df[df['Addresses'].isna()]
nonedata

,HotspotID,LabName,Addresses
68,87,"Madurai Medical College, Madurai",None
76,96,"Madras Medical College, Chennai, Tamil Nadu",None
79,100,"Government Villupuram Medical College, Villupuram",None
81,102,"Government Medical College, Thiruvarur",None
106,130,"ICMR-National Institute of Virology, Pune",None


In [10]:
df['LabName'].isna().sum()

0

In [11]:
for i in [68,76,79,81,106]:
    soup = BeautifulSoup(pages[i].content, 'html.parser')
    all_hotspots = soup.find_all("div" ,class_ ='hotspots-description')
    print(all_hotspots[0].text[7:-9])

l College, Medical College Rd, Alwarpuram, Madurai, Tamil Nad
Madras Medical College, Near Park Town Station, Park Town, Chennai, Tamil Nad
Government Villupuram Medical College, Mundiyampakkam, Villupuram, Tamil Nad
Government Medical College, Collector office Master Plan Complex, Collector Office Road, Thiruvarur, Tamil Nad
al Institute of Virology, Pune, 130/1, Pashan - Sus Rd, Pashan Gaon, Pashan, Pune, Maharashtr


In [12]:
df['Addresses'][68] =  'Medical College, Medical College Rd, Alwarpuram, Madurai, Tamil Nadu 625020'
df['Addresses'][106] = 'National Institute of Virology, Pune, 130/1, Pashan - Sus Rd, Pashan Gaon, Pashan, Pune, Maharashtra 411021'

In [13]:
nonedata = df[df['Addresses'].isna()]
nonedata

,HotspotID,LabName,Addresses
76,96,"Madras Medical College, Chennai, Tamil Nadu",None
79,100,"Government Villupuram Medical College, Villupuram",None
81,102,"Government Medical College, Thiruvarur",None


In [14]:
for i in [76,79,81]:
    soup = BeautifulSoup(pages[i].content, 'html.parser')
    all_hotspots = soup.find_all("div" ,class_ ='hotspots-description')
    df['Addresses'][i]=all_hotspots[0].text[7:]

In [15]:
df.isna().sum()

HotspotID    0
LabName      0
Addresses    0
dtype: int64

In [17]:
df.to_csv('IndianCOVID_testCentres_raw.csv',index=False)